In [18]:
from typing import Any, Dict
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate, FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

prompt.format(country="Germany")



# examples = [
#     {
#         "question": "What do you know about France?",
#         "answer": """
#         Here is what I know:
#         Capital: Paris
#         Language: French
#         Food: Wine and Cheese
#         Currency: Euro
#         """,
#     },
#     {
#         "question": "What do you know about Italy?",
#         "answer": """
#         I know this:
#         Capital: Rome
#         Language: Italian
#         Food: Pizza and Pasta
#         Currency: Euro
#         """,
#     },
#     {
#         "question": "What do you know about Greece?",
#         "answer": """
#         I know this:
#         Capital: Athens
#         Language: Greek
#         Food: Souvlaki and Feta Cheese
#         Currency: Euro
#         """,
#     },
# ]

# class RandomExampleSelector(BaseExampleSelector):

#     def __init__(self, examples):
#         self.examples = examples

#     def add_example(self, example):
#         self.examples.append(example)

#     def select_examples(self, input_variables):
#         from random import choice
#         return [choice(self.examples)]


# example_prompt = PromptTemplate.from_template(
#     ("Human: {question}\nAI:{answer}")
# )

# example_selector = RandomExampleSelector(
#     examples=examples,
# )

# prompt = FewShotPromptTemplate(
#     example_prompt=example_prompt,
#     example_selector=example_selector,
#     suffix="Human: What do you konw about {country}?",
#     input_variables=["country"]
# )

# prompt.format(country="Canada")

'What is the capital of Germany'